# Transformers

To understand transformers from scratch, refer [this]('http://www.peterbloem.nl/blog/transformers') blog.


In [1]:
# IMPORTING LIBRARIES
import torch 
from torch import nn
from torch.nn import LayerNorm,Dropout,Linear,Softmax
# import activation function named softmax
import torch.nn.functional as F
from torch.nn import Embedding
import torch.nn.functional as F
from sklearn.metrics import f1_score
import torchtext
from torchtext import datasets,data
from torchtext.data import Field # Field is used to further generate mapping ids of each word using build_vocab fucntion for training data

In [2]:
# Load dataset
import pandas as pd
import numpy as np
train_data = pd.read_csv('../input/trainingtest/cleaned_train_fina.csv')['user_review'].values
labels = list(pd.read_csv('../input/trainingtest/train.csv')['user_suggestion'].values)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train_data,labels,test_size=0.2)
# now generate different csvs for trianing and validation data
# save both train_csv and test_csv with .csv extension


In [3]:
len(x_test)

3499

In [4]:
train_csv = pd.DataFrame({'user_review':x_train,'user_suggestion':y_train})
test_csv = pd.DataFrame({'user_review':x_test,'user_suggestion':y_test})

In [11]:
# create datafield for text and label
TEXT = data.Field(sequential=True,include_lengths=True,batch_first=True,lower=True)
LABEL = data.Field(sequential=False)

train,val = data.TabularDataset.splits(format='csv',fields=[('text',TEXT),('label',LABEL)],path='../input/torchtext2/',train='train (2).csv',validation='val.csv',skip_header=True)
# now write BucketIterator for train and val dataset
TEXT.build_vocab(train,max_size=17494)
LABEL.build_vocab(train)
train_iter,val_iter = data.BucketIterator.splits(datasets=(train,val),batch_size=32,device=torch.device('cpu'),sort=False,sort_key = lambda x:len(x.user_review),sort_within_batch=False,repeat=False)


In [12]:
len(train_iter)

438

In [13]:
len(TEXT.vocab)

17496

In [14]:
import tqdm
len(val_iter)

110

In [15]:
for b in train_iter:
    print(b.label-1,type(b.text[0]))
    print(b.text[0].tolist()[31][:100])
    print(torch.LongTensor(b.text[0]))
    ls = [b.text[0].tolist()[x][:50] for x in range(32)]
    

tensor([0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
        0, 1, 1, 1, 1, 1, 0, 1]) <class 'torch.Tensor'>
[567, 647, 1131, 14, 11577, 3, 1416, 1618, 5, 14801, 9, 18, 5, 148, 2, 1110, 218, 4, 2617, 116, 2, 276, 5, 980, 0, 2190, 840, 1659, 12, 1097, 1924, 136, 1060, 143, 335, 2, 3650, 324, 233, 4000, 6, 5897, 2, 425, 2, 14308, 1438, 226, 1575, 43, 1342, 1300, 18, 18, 2, 2037, 792, 207, 2581, 746, 2, 84, 854, 52, 1741, 278, 1156, 2, 3169, 603, 1235, 10, 243, 2, 754, 358, 2, 149, 2, 2191, 261, 253, 1227, 398, 1382, 886, 49, 2558, 2404, 1150, 108, 396, 138, 1214, 3, 2, 2387, 2, 664, 3]
tensor([[4552,  382,   19,  ...,    1,    1,    1],
        [  17,   14,  223,  ...,    1,    1,    1],
        [  17,   14,  223,  ...,    1,    1,    1],
        ...,
        [  17,   14, 2341,  ...,    1,    1,    1],
        [   4,   64,  502,  ...,  307,  962,   30],
        [ 567,  647, 1131,  ...,    1,    1,    1]])
tensor([0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 

tensor([0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 1]) <class 'torch.Tensor'>
[84, 35, 3, 2, 4, 696, 7, 2, 19, 442, 3247, 184, 775, 1064, 434, 155, 51, 0, 1150, 12, 248, 2282, 446, 432, 2, 245, 2759, 2, 28, 11, 308, 1238, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tensor([[  84,   31,   48,  ...,    1,    1,    1],
        [  17,   14,  758,  ...,    1,    1,    1],
        [  17,   14, 6365,  ...,    1,    1,    1],
        ...,
        [1092,  454,  299,  ..., 1469, 3573,  128],
        [  17,   14, 9120,  ...,    1,    1,    1],
        [  84,   35,    3,  ...,    1,    1,    1]])
tensor([1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 0, 0, 0, 0]) <class 'torch.Tensor'>
[162, 272, 2151, 4169, 2, 3, 271, 2, 182, 527, 1866, 4

tensor([1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
        0, 1, 1, 1, 0, 0, 1, 1]) <class 'torch.Tensor'>
[17, 14, 107, 1159, 6, 1290, 260, 2190, 373, 2, 4, 266, 3588, 537, 4387, 2, 83, 102, 2, 924, 1280, 77, 31, 226, 1204, 198, 3, 2, 1655, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tensor([[   74,     4,     0,  ...,     1,     1,     1],
        [   45,   286,   374,  ...,     1,     1,     1],
        [   17,    14, 13979,  ...,     1,     1,     1],
        ...,
        [    4,    64,    27,  ...,     1,     1,     1],
        [   81,     5,    16,  ...,     1,     1,     1],
        [   17,    14,   107,  ...,     1,     1,     1]])
tensor([0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 1, 0, 0, 0]) <class 'torch.Tensor'>
[3590, 186, 1983, 

tensor([1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 1, 0, 0, 0]) <class 'torch.Tensor'>
[1057, 477, 1606, 0, 2, 1502, 2, 3929, 292, 1596, 2, 304, 43, 2, 92, 31, 5190, 6040, 109, 754, 2070, 164, 2, 807, 194, 2, 10, 0, 8436, 2, 2021, 1502, 1113, 126, 0, 3717, 3718, 2, 4, 73, 1349, 40, 2, 16, 3, 9, 37, 942, 119, 12, 248, 1588, 578, 424, 1325, 8452, 1811, 324, 141, 2, 12, 5, 230, 2, 930, 9384, 36, 2489, 23, 12, 5, 128, 499, 1480, 772, 0, 9475, 3065, 71, 4052, 697, 169, 98, 41, 163, 206, 2, 1863, 99, 505, 1353, 43, 2, 1481, 84, 4, 36, 2, 4412, 405]
tensor([[1281,   51,   66,  ...,    1,    1,    1],
        [4453,    2,   51,  ...,    1,    1,    1],
        [  17,   14, 5341,  ...,    1,    1,    1],
        ...,
        [ 254,  113,  179,  ...,    1,    1,    1],
        [   4,   65,    3,  ...,    1,    1,    1],
        [1057,  477, 1606,  ...,    1,    1,    1]])
tensor([0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,

tensor([0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
        0, 1, 0, 1, 1, 1, 1, 0]) <class 'torch.Tensor'>
[16, 3, 1486, 72, 2, 561, 1666, 679, 416, 846, 2, 10, 0, 155, 335, 2, 4, 34, 8, 25, 4316, 562, 1334, 36, 10268, 14166, 11397, 2, 363, 858, 5861, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tensor([[ 421,    4,  581,  ...,    1,    1,    1],
        [ 562,  171,    3,  ...,    1,    1,    1],
        [  17,   14,  223,  ...,    1,    1,    1],
        ...,
        [ 251,   27,  348,  ...,    1,    1,    1],
        [2020, 1376,    2,  ...,    1,    1,    1],
        [  16,    3, 1486,  ...,    1,    1,    1]])
tensor([1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 0]) <class 'torch.Tensor'>
[16, 342, 9, 2, 49, 0, 2, 4, 41, 0, 2, 10, 0, 442,

...

tensor([0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
        1, 0, 0, 1, 1, 0, 0, 1]) <class 'torch.Tensor'>
[1094, 279, 3391, 25, 1054, 1544, 2, 4, 53, 329, 2046, 0, 230, 159, 6394, 86, 377, 572, 1563, 2, 1492, 3331, 2803, 5255, 456, 40, 6, 456, 0, 496, 2, 111, 381, 24, 1893, 2025, 2, 3, 434, 2111, 918, 627, 137, 7865, 2, 1671, 497, 29, 8, 65, 3, 2, 196, 242, 1356, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tensor([[  21,   88,    4,  ...,    1,    1,    1],
        [  16,   20,  246,  ...,    1,    1,    1],
        [  45,    3,  744,  ...,    1,    1,    1],
        ...,
        [  17,   14, 7218,  ...,    1,    1,    1],
        [  84,   92,  244,  ...,    1,    1,    1],
        [1094,  279, 3391,  ...,    1,    1,    1]])
tensor([0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 1, 0, 0, 0, 1]) <class 'torch.Tensor'>
[17, 14

tensor([0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 1, 0, 0, 0, 1, 1]) <class 'torch.Tensor'>
[17, 14, 223, 321, 5, 474, 9, 22, 24, 2, 180, 3, 2, 39, 45, 5, 3, 252, 12, 1616, 298, 207, 2, 901, 2402, 5810, 6530, 1796, 2, 5700, 2793, 2, 212, 586, 779, 1426, 1218, 195, 430, 2107, 2, 294, 592, 338, 1194, 2332, 1029, 819, 277, 2, 4, 574, 77, 69, 260, 6, 253, 217, 31, 48, 2, 1846, 19, 82, 48, 807, 2, 129, 445, 6673, 0, 2281, 121, 1502, 58, 224, 27, 34, 8, 672, 21, 2, 28, 11, 870, 2, 1037, 2, 34, 78, 505, 763, 709, 2, 1157, 7, 97, 48, 1, 1]
tensor([[  20,    3, 2955,  ...,    1,    1,    1],
        [  35,    3,    2,  ...,    1,    1,    1],
        [  17,   14, 6761,  ...,    1,    1,    1],
        ...,
        [  17,   14, 9121,  ...,    1,    1,    1],
        [   4,   64,  494,  ...,    1,    1,    1],
        [  17,   14,  223,  ...,    1,    1,    1]])
tensor([1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1,
        0, 1

tensor([[  17,   14,  223,  ...,    1,    1,    1],
        [  17,   14,  107,  ...,    1,    1,    1],
        [  10,    3,   71,  ...,    1,    1,    1],
        ...,
        [ 656,    4, 2945,  ...,    1,    1,    1],
        [  29,    8,   13,  ...,    1,    1,    1],
        [ 567,  647, 1131,  ...,    1,    1,    1]])
tensor([1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 0, 0, 0, 1, 1, 1, 0]) <class 'torch.Tensor'>
[17, 14, 0, 91, 80, 117, 2, 45, 220, 2, 237, 950, 3722, 1795, 408, 58, 1582, 3, 2, 1063, 192, 917, 2, 56, 0, 34, 861, 243, 596, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tensor([[  17,   14,  223,  ...,    1,    1,    1],
        [ 744, 2492,  253,  ...,    1,    1,    1],
        [  17,   14, 1133,  ...,    1,    1,    1],
        ...,
        [ 284,   27,  751,  

tensor([0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
        1, 1, 0, 0, 1, 0, 0, 0]) <class 'torch.Tensor'>
[41, 3, 98, 99, 1439, 345, 251, 2, 20, 171, 3, 2, 8, 216, 36, 2734, 3459, 12, 2268, 2, 4, 23, 905, 200, 179, 2, 7, 11, 51, 66, 19, 203, 146, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tensor([[   7,   11,   35,  ...,    1,    1,    1],
        [  35,    3,    2,  ...,    1,    1,    1],
        [  16,  107,  706,  ...,    1,    1,    1],
        ...,
        [   7,   11,  957,  ...,    1,    1,    1],
        [  17,   14, 6762,  ...,    1,    1,    1],
        [  41,    3,   98,  ...,    1,    1,    1]])
tensor([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
        1, 1, 0, 0, 0, 1, 1, 0]) <class 'torch.Tensor'>
[4, 64, 3518, 3, 197, 862, 555, 4, 26, 1043, 175, 2, 41, 190

IndexError: list index out of range

In [16]:
len(ls)

32

In [17]:
import pandas as pd
import numpy as np
train_data = pd.read_csv('../input/trainingtest/cleaned_train_fina.csv')['user_review'].values
labels = list(pd.read_csv('../input/trainingtest/train.csv')['user_suggestion'].values)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train_data,labels,test_size=0.2)
train_data = np.array(train_data)
train_data = list(train_data)
labels = list(y_train)

test_data = train_data
print(test_data[0])
final_data = []
import re
for data in test_data:
    new = re.sub('[0-9]','',str(data))
    final_data.append(new)
print(final_data[0])
from tensorflow.keras.preprocessing import text,sequence

tokenize = text.Tokenizer()

tokenize.fit_on_texts(texts = (final_data))
index_data = tokenize.texts_to_sequences(final_data)

word_index = tokenize.word_index

i m scared hearing creepy voices  so i will pause moment write review i wait heart beat return atleast somewhat calmer times  this game adorable like happy tree friends graphics sceme childhood  but bubble  clean    hello 1990 s what charactes  that isnot trying kill me  likable bit odd  noob things though  as oh look class room full ghosts dead children  let us shine flashlight stand staring them  or  hmm music  turn around see what is chasing me never afraid finding locked door 
i m scared hearing creepy voices  so i will pause moment write review i wait heart beat return atleast somewhat calmer times  this game adorable like happy tree friends graphics sceme childhood  but bubble  clean    hello  s what charactes  that isnot trying kill me  likable bit odd  noob things though  as oh look class room full ghosts dead children  let us shine flashlight stand staring them  or  hmm music  turn around see what is chasing me never afraid finding locked door 


In [18]:
# Self Attention Architecture
class selfattention(nn.Module):
    
    def __init__(self,k,heads=9):
        super().__init__()
        self.k , self.heads = k,heads
        # initialize dimensions for keys,values and queries as we are building multi-head attention layers for modern transformer
        self.tokeys = nn.Linear(k,k*heads,bias=False)
        self.toqueries = nn.Linear(k,k*heads,bias=False)
        self.tovalues = nn.Linear(k,k*heads,bias=False)
        
        self.unified = nn.Linear(k*heads,k,bias=False)
        
    def forward(self,x):  # pass values into __init__ function
        b,t,k = x.size()  # where b is number of batches, t is size of input sequence length and k is number of dimensions
        # now determine keys,values,queries and heads
        h = self.heads
        
        keys = self.tokeys(x).view(b,t,h,k) # initially we don't have head next to each other
        queries = self.toqueries(x).view(b,t,h,k)
        values = self.tovalues(x).view(b,t,h,k)
    
        # now we want head next to batch, its highly computational expensive but we have to do it, so transform keys,values and queries
        keys = keys.transpose(1,2).contiguous().view(b*h,t,k)
        queries = queries.transpose(1,2).contiguous().view(b*h,t,k)
        values = values.transpose(1,2).contiguous().view(b*h,t,k)
        # now calculate dot product using bmm function in pytorch
        dot = torch.bmm(queries,keys.transpose(1,2))
        
        dot = F.softmax(dot,dim=2)
        out = torch.bmm(dot,values).view(b,h,t,k)
        
        # now finally we want unified output in k dimensions as we have initially, so to do this again transspose out
        out = out.transpose(1,2).contiguous().view(b,t,h*k)
        out = self.unified(out)
        return out  

In [19]:
# now build architecture for transformer block
class transformer(nn.Module):
    
    def __init__(self,k,heads=9):
        super().__init__()
        self.attention = selfattention(k) # pass class named selfattention
        # now add normalization layer to normalize outputs of attention layer
        self.norm1 = nn.LayerNorm(k)
        self.norm2 = nn.LayerNorm(k)
        # now make a fully connected multi layer perceptron
        self.ff = nn.Sequential(nn.Linear(k,4*k),nn.ReLU(),nn.Linear(4*k,k)) # fully connected layer for hidden states
        self.drop = nn.Dropout(0.5)
    
    def forward(self,x):
        attention = self.attention(x)
        x = self.norm1(attention+x)
        x = self.drop(x) # dropout layer after normalization(drop some neurons to prevent from overfitting)
        perceptron = self.ff(x)
        x = self.norm2(perceptron + x)
        x = self.drop(x)
        return x

In [34]:
# now add classification and add embedding layer initially to it.
# REMEMBER INPUT SHOULD BE INDEXES OF WORDS 
class classify(nn.Module):
    
    def __init__(self,k,seq_length,num_tokens,depth,num_classes,max_pool=True,heads=9):
        super().__init__()
        # initialize tokens
        self.num_tokens = num_tokens
        self.maxpool = max_pool
        # it needs input in word indexes using tokenizer
        self.tokenemb = Embedding(embedding_dim=k,num_embeddings=num_tokens)
        # position embedding
        self.posemb = Embedding(embedding_dim=k, num_embeddings=seq_length)
        tfblocks = []
        for i in range(depth):
            tfblocks.append(transformer(k))
        # now add sequential layer of tfblocks
        self.transform = nn.Sequential(*tfblocks)
        # add linear layer to convert into desired number of class i.e. 2 for binary class
        self.prob = nn.Linear(k,num_classes)
        self.drop = nn.Dropout(0.5)
        
    def forward(self,x,y):
        tokens = self.tokenemb(x)
        b,t,k = tokens.size()
        for batch in range(b):
            for pos in range(t):
                for i in range(k):
                    if i%2==0:
                        tokens[batch][pos][i]+=np.sin(pos/(1000**(2*i/k)))
                    else:
                        tokens[batch][pos][i]+=np.cos(pos/(1000**(((2*i)+1)/k)))
        #positions = self.posemb(torch.arange(t, device=torch.device('cuda')))[None, :, :].expand(b, t, k)
        
        x = tokens # adding tokens and positions to get proper embeddings of each indexes of words
        x = self.drop(x)
        x = self.transform(x)
        x = x.max(dim=1)[0] if self.maxpool else x.mean(dim=1)
        x = self.prob(x)
        x = F.softmax(x,dim=1)
        loss = torch.nn.CrossEntropyLoss()
        loss = loss(x,y)
        return loss,x        

In [22]:
device=torch.device('cpu')

In [35]:
# initializing classify model for binary classification
classifier = classify(128,100,17496,12,2)
classifier.to(device)

classify(
  (tokenemb): Embedding(17496, 128)
  (posemb): Embedding(100, 128)
  (transform): Sequential(
    (0): transformer(
      (attention): selfattention(
        (tokeys): Linear(in_features=128, out_features=1152, bias=False)
        (toqueries): Linear(in_features=128, out_features=1152, bias=False)
        (tovalues): Linear(in_features=128, out_features=1152, bias=False)
        (unified): Linear(in_features=1152, out_features=128, bias=False)
      )
      (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (ff): Sequential(
        (0): Linear(in_features=128, out_features=512, bias=True)
        (1): ReLU()
        (2): Linear(in_features=512, out_features=128, bias=True)
      )
      (drop): Dropout(p=0.5, inplace=False)
    )
    (1): transformer(
      (attention): selfattention(
        (tokeys): Linear(in_features=128, out_features=1152, bias=False)
        (toqueries): Linear(in_f

In [36]:
#pad_sequences_train,pad_sequences_test = train_test_split(pad_sequences,test_size=0.2)

In [25]:
#from torch.utils.data import DataLoader,SequentialSampler,TensorDataset,RandomSampler,DistributedSampler,BatchSampler

In [26]:
#dataset = torch.utils.data.TensorDataset(torch.LongTensor(pad_sequences_train), torch.LongTensor(labels))

In [27]:
#batch_data = DataLoader(batch_size=128,dataset = dataset,sampler = SequentialSampler(dataset))

In [28]:
'''y_test = np.array(y_test)
y_test = list(y_test)

test_dataset = TensorDataset(torch.LongTensor(pad_sequences_test),torch.LongTensor(y_test))
test_loader = DataLoader(test_dataset,batch_size=128,sampler = RandomSampler(test_dataset))'''

'y_test = np.array(y_test)\ny_test = list(y_test)\n\ntest_dataset = TensorDataset(torch.LongTensor(pad_sequences_test),torch.LongTensor(y_test))\ntest_loader = DataLoader(test_dataset,batch_size=128,sampler = RandomSampler(test_dataset))'

In [29]:
# declare optimizer 
from torch.autograd import no_grad,Variable
from torch.optim import Optimizer
from torch.optim import Adam,SGD,AdamW,lr_scheduler
optimizer = Adam(classifier.parameters(),lr=0.03,amsgrad=True)

In [30]:
''''x_test = np.array(x_test)
x_test = list(x_test)
y_test = np.array(y_test)
y_test = list(y_test)
test_data = x_test
final_data_test = []
# generating padded sequences for test data
import re
for data in test_data:
    new = re.sub('[0-9]','',str(data))
    final_data_test.append(new)
print(final_data_test[0])
from tensorflow.keras.preprocessing import text,sequence

tokenize = text.Tokenizer()

tokenize.fit_on_texts(texts = (final_data_test))
index_data_test = tokenize.texts_to_sequences(final_data_test)

word_index_test = tokenize.word_index
print(len(word_index_test))

#pad_sequences_test = sequence.pad_sequences(index_data_test,maxlen=50)

for test in test_loader:
    print(test)
    break'''

"'x_test = np.array(x_test)\nx_test = list(x_test)\ny_test = np.array(y_test)\ny_test = list(y_test)\ntest_data = x_test\nfinal_data_test = []\n# generating padded sequences for test data\nimport re\nfor data in test_data:\n    new = re.sub('[0-9]','',str(data))\n    final_data_test.append(new)\nprint(final_data_test[0])\nfrom tensorflow.keras.preprocessing import text,sequence\n\ntokenize = text.Tokenizer()\n\ntokenize.fit_on_texts(texts = (final_data_test))\nindex_data_test = tokenize.texts_to_sequences(final_data_test)\n\nword_index_test = tokenize.word_index\nprint(len(word_index_test))\n\n#pad_sequences_test = sequence.pad_sequences(index_data_test,maxlen=50)\n\nfor test in test_loader:\n    print(test)\n    break"

In [31]:
# set scheduler for learning rate, for that calculate total steps
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import RandomSampler
epochs = 100 # researchers suggest to take epochs should be in range of 4-7 for fine tuning pretrained model as we have concern 
# just for last layer which is untrained classification layer.
total_steps = len(train_iter) * epochs
# scheduler take care of linear schedule of learning rate 
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=total_steps)

In [32]:
device = torch.device('cpu')

In [ ]:
# Training 
epochs = 100
final_loss = []
output = []
testing_accuracy = []
final_test_loss=[]
for epoc in range(epochs):
    print('epoch-',epoc)
    total_loss = 0
    classifier.train()
    for batch in train_iter:
        optimizer.zero_grad()
        # now check length of input sequence whether it is greater than 512 or not
        if batch.text[0].size(1)>50:
            batch_train = batch.text[0].tolist()
            batch_train = [batch_train[x][:50] for x in range(len(batch_train))]
            
        else:
            batch_train=batch.text[0].tolist()
        loss,outputs = classifier.forward(torch.LongTensor(batch_train).to(device),(batch.label-1).to(device)) # here -2 is because of padding tokens

        loss.backward()
        total_loss+=loss.item()
        torch.nn.utils.clip_grad_norm_(classifier.parameters(),1.0)
        optimizer.step()
        scheduler.step()
    avg_loss = total_loss/len(train_iter)
    print('train_loss',avg_loss)
    final_loss.append(avg_loss) # testing outputs
    # Validating
    classifier.eval()
    test_accuracy = 0
    test_loss=0
    for batch_t in val_iter:
        with torch.no_grad():       
            if batch_t.text[0].size(1)>50:
                batch_t_test = batch_t.text[0].tolist()
                batch_t_test = [batch_t_test[x][:50] for x in range(len(batch_t_test))]
            else:
                batch_t_test= batch_t.text[0].tolist()
            outpu = classifier.forward(torch.LongTensor(batch_t_test).to(device),(batch_t.label-1).to(device))
            predictions = outpu[1]
            # no need for backward loss and zero grad
            test_accuracy += f1_score(y_pred=np.argmax((predictions.cpu().detach().numpy()),axis=1),y_true=(batch_t.label-1).cpu().detach().numpy())
            test_loss+=outpu[0].item()
            output.append(predictions)
    avg_test_loss = test_loss/len(val_iter)
    print('val_loss',avg_test_loss)
    final_test_loss.append(avg_test_loss)
    avg_accuracy = test_accuracy/len(val_iter)
    testing_accuracy.append(avg_accuracy)    
    

epoch- 0


In [ ]:
# saving entire model
torch.save(classifier,'tf_classify.pth')

In [ ]:
predictions

In [ ]:
import matplotlib.pyplot as plt
ep = []
for e in range(epoc):
    ep.append(e)
plt.plot(ep,final_loss)
plt.plot(ep,final_test_loss)
plt.savefig('epochs_vs_loss')

In [ ]:
final_loss